In [1]:
import pandas as pd
import glob
import os
import numpy as np
import random

import torch
from torch import nn
from torch.utils.data import DataLoader
from itertools import combinations,permutations

torch.manual_seed(99)
np.random.seed(99)
torch.cuda.empty_cache()
import torch.nn.functional as F

import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as tf

from enum import Enum
from typing import List

import pandas as pd
import pkg_resources
import torch
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from torch.utils.data import TensorDataset, DataLoader

import networkx as nx
from tqdm import tqdm
from torch_geometric.nn import GCNConv, GATConv
import plotly.graph_objects as go
from scipy.sparse import coo_matrix
from geopy.distance import distance

In [2]:
# frame = pd.read_csv("C:/Users/psheth5/STCD-RL/data/data/watershed_avg.csv")
frame = pd.read_csv("C:/Users/psheth5/STCD-RL/data/data/StandardizedData.csv")
n_features = frame.shape[1]
frame

,-95.35 28.85 \n,-103.593750 34.656250\n,-103.531250 34.656250\n,-103.468750 34.656250\n,-103.406250 34.656250\n,-103.343750 34.656250\n,-103.281250 34.656250\n,-103.218750 34.656250\n,-103.156250 34.656250\n,-103.656250 34.593750\n,...,-95.531250 29.031250\n,-95.468750 29.031250\n,-95.406250 29.031250\n,-95.531250 28.968750\n,-95.468750 28.968750\n,-95.406250 28.968750\n,-95.343750 28.968750\n,-95.468750 28.906250\n,-95.406250 28.906250\n,-95.343750 28.906250\n
0,9.911094,-0.134267,-0.398994,0.256755,-0.383672,1.572451,-0.674415,-0.228816,0.037667,-0.532050,...,-0.178725,-0.230239,-0.252408,-0.572311,0.088742,0.446975,-0.651768,-0.300383,0.901711,0.067492
1,0.035373,-0.726517,-0.466346,-0.129167,-0.578796,0.337814,-0.375985,-0.711423,-0.462730,-0.125976,...,-0.337374,-0.346014,-0.374297,-0.734310,-0.044489,0.021120,-0.418514,-0.612205,0.086772,-0.196638
2,-0.184683,-0.176776,-0.466799,-0.400074,-0.427399,-0.359177,-0.275025,-0.393831,-0.511412,-0.347567,...,-0.329134,-0.343043,-0.378763,-0.592142,-0.343849,-0.497109,-0.257276,-0.389134,-0.491061,-0.162036
3,-0.184683,-0.387886,-0.594697,-0.516747,-0.469316,-0.495481,-0.543043,-0.590618,-0.496078,-0.469463,...,-0.337176,-0.345840,-0.380737,-0.606311,-0.482309,-0.705154,-0.588362,-0.593416,-0.691268,-0.405932
4,0.019306,-0.445776,-0.633418,-0.487968,-0.469061,-0.491576,-0.560085,-0.603027,-0.560102,-0.470457,...,-0.337135,-0.345820,-0.381714,-0.576330,-0.507051,-0.716281,-0.573016,-0.623524,-0.713250,-0.414172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3511,-0.115697,1.059131,0.583768,-0.293875,0.527452,0.691676,-0.052087,0.410897,0.046137,0.737526,...,2.517924,3.742895,3.391589,0.305562,0.104015,0.160366,-0.235124,0.201521,1.091970,0.155848
3512,-0.183097,0.716132,0.258574,-0.420926,0.113760,0.450778,-0.237647,0.160923,-0.077343,0.466684,...,0.530899,1.000613,0.895689,0.052787,-0.114444,0.007075,-0.462386,-0.039817,0.773744,-0.054653
3513,-0.183720,0.696043,0.269436,-0.401614,0.077996,0.506875,-0.108806,0.183974,-0.030824,0.535118,...,-0.126830,-0.057985,-0.049533,-0.042148,-0.045282,0.017888,-0.357489,0.067339,0.832164,-0.098938
3514,-0.144083,0.481942,0.191978,-0.412277,0.165111,0.442213,-0.115928,0.194667,0.011004,0.544686,...,-0.344364,-0.323794,-0.296564,0.005189,0.174514,0.168020,-0.398325,-0.001075,0.757593,-0.133510


In [3]:
# import numpy as np

# adj_mat = np.load('C:/Users/psheth5/STCD-RL/data/data/watershed.npy')

adj_mat = np.load('ElevationAdjacency.npy')


In [ ]:
# # distances = np.load('C:/Users/psheth5/STCD-RL/distance.npy')
# # np.fill_diagonal(distance, np.inf)
# # distances[np.isfinite(distance)] = 1   # replace finite values with 1
# # distances[~np.isfinite(distance)] = 0   # replace infinite values with 0
# # distances

# list_of_coords_str = frame.columns # your list of 3129 latitude longitude values
# list_of_coords = [tuple(map(float, c.split()[::-1])) for c in list_of_coords_str]
# adj_matrix_list = np.zeros((len(list_of_coords), len(frame.columns)))
# for i, c1 in enumerate(list_of_coords):
#     for j, c2 in enumerate(list_of_coords):
#         if c2[0] > c1[0] and distance(c1, c2).km > 0:
#             adj_matrix_list[i][j] = 1

In [4]:
# # adj_mat = np.load('C:/Users/psheth5/STCD-RL/data/data/watershed.npy')
# coo = coo_matrix(adj_matrix_list, dtype = "int8")
coo = coo_matrix(adj_mat, dtype = "int8")
row = torch.from_numpy(coo.row.astype(np.int64)).to(torch.long)
col = torch.from_numpy(coo.col.astype(np.int64)).to(torch.long)
edge_index = torch.stack([row, col], dim=0)

In [ ]:
# np.save("data\\data\\adj_mat_new.npy", adj_matrix_list)

In [5]:
import numpy as np
import torch

def filter_edge_index(edge_index, coords):
    filtered_index = []
    for i in range(edge_index.shape[1]):
        u = edge_index[0][i]
        v = edge_index[1][i]
        # check if u is above v
        if torch.tensor(coords[u][0]) > torch.tensor(coords[v][0]) and u != v:
            filtered_index.append([u, v])
    filtered_index = np.array(filtered_index).T
    return filtered_index

In [6]:
# edge_index = filter_edge_index(edge_index, list_of_coords)
# edge_index = torch.from_numpy(edge_index.astype(np.int64)).to(torch.long)

In [7]:
# create a mapping function
def map_values(value, value_range):
    """Maps a value from the input range to the output range."""
    input_min, input_max = value_range
    output_min, output_max = (0, 99)
    return torch.floor(((value - input_min) / (input_max - input_min)) * (output_max - output_min) + output_min).long()


def reverse_map_values(mapped_value, value_range):
    """Maps a value from the output range to the input range."""
    input_min, input_max = value_range
    output_min, output_max = (0, 99)
    return torch.floor(((mapped_value - output_min) / (output_max - output_min)) * (input_max - input_min) + input_min).long()

In [8]:
class Tasks(Enum):
    prediction = "prediction"
    reconstruction = "reconstruction"


class TimeSeriesDataset(object):
    def __init__(self, task: Tasks, data_path: str, categorical_cols: List[str], index_col: str, target_col: str,
                 seq_length: int, batch_size: int, prediction_window: int = 1):
        """
        :param task: name of the task
        :param data_path: path to datafile
        :param categorical_cols: name of the categorical columns, if None pass empty list
        :param index_col: column to use as index
        :param target_col: name of the targeted column
        :param seq_length: window length to use
        :param batch_size:
        :param prediction_window: window length to predict
        """
        self.task = task.value

        

        # data_path = pkg_resources.resource_filename("tsa", data_path)
        self.data = pd.read_csv(data_path, index_col=index_col)[0:1460]
        # a  = [self.data.columns[0]]
        # a.extend(self.data.columns[-250:])
        # self.data = self.data[a]
        self.categorical_cols = categorical_cols
        # self.numerical_cols = list(set(self.data.columns) - set(categorical_cols) - set(target_col))
        self.numerical_cols = self.data.columns
        print(len(self.numerical_cols))
        self.target_col = target_col

        self.seq_length = seq_length
        self.prediction_window = prediction_window
        self.batch_size = batch_size

        transformations = [("scaler", StandardScaler(), self.numerical_cols)]
        if len(self.categorical_cols) > 0:
            transformations.append(("encoder", OneHotEncoder(), self.categorical_cols))
        self.preprocessor = ColumnTransformer(transformations, remainder="passthrough")

        if self.task == "prediction":
            self.y_scaler = StandardScaler()

    def preprocess_data(self):
        """Preprocessing function"""
        X = self.data#.drop(self.target_col, axis=1,inplace=False)
        # y = self.data[self.target_col]

        X_train, X_test = train_test_split(X,train_size=0.8, shuffle=False)
        X_train = self.preprocessor.fit_transform(X_train)
        X_test = self.preprocessor.transform(X_test)

        # if self.task == "prediction":
        #     y_train = self.y_scaler.fit_transform([y_train])
        #     y_test = self.y_scaler.fit_transform([y_test])
        #     return X_train, X_test, y_train.reshape(-1,1), y_test.reshape(-1,1)
        return X_train, X_test, None,None

    def frame_series(self, X, y=None):
        """
        Function used to prepare the data for time series prediction
        :param X: set of features
        :param y: targeted value to predict
        :return: TensorDataset
        """
        nb_obs, nb_features = X.shape
        features, target, y_hist = [], [], []

        for i in range(1, nb_obs - self.seq_length - self.prediction_window):
            features.append(torch.FloatTensor(X[i:i + self.seq_length, :]).unsqueeze(0))

            if self.task == "prediction":
                # lagged output used for prediction
                y_hist.append(torch.FloatTensor(y[i - 1:i + self.seq_length - 1]).unsqueeze(0))
                # shifted target
                target.append(torch.FloatTensor(y[i + self.seq_length:i + self.seq_length + self.prediction_window]).unsqueeze(0))
            else:
                y_hist.append(torch.FloatTensor(X[i - 1: i + self.seq_length - 1, :]).unsqueeze(0))
                target.append(
                    torch.FloatTensor(X[i + self.seq_length:i + self.seq_length + self.prediction_window, :]))

        features_var = torch.cat(features)
        y_hist_var = torch.cat(y_hist)
        target_var = torch.cat(target)

        return TensorDataset(features_var, y_hist_var, target_var)

    def get_loaders(self):
        """
        Preprocess and frame the dataset

        :return: DataLoaders associated to training and testing data
        """
        X_train, X_test, y_train, y_test = self.preprocess_data()
        nb_features = X_train.shape[1]

        train_dataset = self.frame_series(X_train, y_train)
        test_dataset = self.frame_series(X_test, y_test)

        train_iter = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, drop_last=True)
        test_iter = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=True, drop_last=True)
        return train_iter, test_iter, nb_features

    def invert_scale(self, predictions):
        """
        Inverts the scale of the predictions
        """
        if isinstance(predictions, torch.Tensor):
            predictions = predictions.numpy()

        if predictions.ndim == 1:
            predictions = predictions.reshape(-1, 1)

        if self.task == "prediction":
            unscaled = self.y_scaler.inverse_transform(predictions)
        else:
            unscaled = self.preprocessor.named_transformers_["scaler"].inverse_transform(predictions)
        return torch.Tensor(unscaled)

In [9]:
task = Tasks('reconstruction')
data1 = TimeSeriesDataset(task=task,data_path="C:/Users/psheth5/STCD-RL/data/data/StandardizedData.csv",categorical_cols=[],index_col=None,target_col=frame.columns[0],seq_length=30,batch_size=256,prediction_window=1)
train_iter, test_iter, nb_features = data1.get_loaders()

3129


In [10]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'


def init_hidden(x: torch.Tensor, hidden_size: int, num_dir: int = 1, xavier: bool = True):
    """
    Initialize hidden.

    Args:
        x: (torch.Tensor): input tensor
        hidden_size: (int):
        num_dir: (int): number of directions in LSTM
        xavier: (bool): wether or not use xavier initialization
    """
    if xavier:
        return nn.init.xavier_normal_(torch.zeros(num_dir, x.size(0), hidden_size)).to(device)
    return Variable(torch.zeros(num_dir, x.size(0), hidden_size)).to(device)


###########################################################################
################################ ENCODERS #################################
###########################################################################
class AttnEncoder(nn.Module):
    def __init__(self, hidden_size_encoder,seq_len,denoising,directions,input_size,edge_index,spatial,use_spatial=False):
        """
        Initialize the network.

        Args:
            config:
            input_size: (int): size of the input
        """
        super(AttnEncoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size_encoder
        self.seq_len = seq_len
        self.add_noise = denoising
        self.gcn_e = GCNConv(in_channels=-1, out_channels=self.hidden_size)
        self.directions = directions
        self.lstm = nn.LSTM(
            input_size=self.input_size,
            hidden_size=self.hidden_size,
            num_layers=1
        )
        self.attn = nn.Linear(
            in_features=2 * self.hidden_size + self.seq_len,
            out_features=1
        )
        self.softmax = nn.Softmax(dim=1)
        self.edge_index=edge_index
        self.use_spatial = use_spatial
        self.spatial_mat = spatial

    def forward(self, input_data: torch.Tensor,sample:torch.Tensor):
        """
        Forward computation.

        Args:
            input_data: (torch.Tensor): tensor of input data
        """
        spatial = self.spatial_mat[sample]
        spatial = spatial.to(device)
        h_t, c_t = (init_hidden(input_data, self.hidden_size, num_dir=self.directions),
                    init_hidden(input_data, self.hidden_size, num_dir=self.directions))

        # apply the mapping function to the sampled tensor
        a = self.edge_index[0][sample]
        b = self.edge_index[1][sample]
        sample_edge_index = torch.stack([a, b], dim=0)
        mapped_arr = map_values(sample_edge_index, (sample_edge_index.min(), sample_edge_index.max()))
        
        attentions, input_encoded = (Variable(torch.zeros(input_data.size(0), self.seq_len, self.input_size)),
                                     Variable(torch.zeros(input_data.size(0), self.seq_len, self.hidden_size)))

        if self.add_noise and self.training:
            input_data += self._get_noise(input_data).to(device)
        
        for t in range(self.seq_len):
            x = torch.cat((h_t.repeat(self.input_size, 1, 1).permute(1, 0, 2),
                           c_t.repeat(self.input_size, 1, 1).permute(1, 0, 2),
                           input_data.permute(0, 2, 1).to(device)), dim=2).to(
                device)  # bs * input_size * (2 * hidden_dim + seq_len)

            e_t = self.attn(x.view(-1, self.hidden_size * 2 + self.seq_len))  # (bs * input_size) * 1
            a_t = self.softmax(e_t.view(-1, self.input_size)).to(device)  # (bs, input_size)
            weighted_input = torch.mul(a_t, input_data[:, t, :].to(device))  # (bs * input_size)
            self.lstm.flatten_parameters()
            _, (h_t, c_t) = self.lstm(weighted_input.unsqueeze(0), (h_t, c_t))

            if(self.use_spatial):
                h_t = self.gcn_e(h_t,mapped_arr.to(device))
            input_encoded[:, t, :] = h_t
            attentions[:, t, :] = a_t




        return attentions, input_encoded


###########################################################################
################################ DECODERS #################################
###########################################################################

class AttnDecoder(nn.Module):
    def __init__(self, seq_len,hidden_size_encoder,hidden_size_decoder,output_size,edge_index):
        """
        Initialize the network.

        Args:
            config:
        """
        super(AttnDecoder, self).__init__()
        self.seq_len = seq_len
        self.encoder_hidden_size = hidden_size_encoder
        self.decoder_hidden_size = hidden_size_decoder
        self.out_feats = output_size

        self.gcn_d = GCNConv(in_channels=-1, out_channels=self.decoder_hidden_size)

        self.attn = nn.Sequential(
            nn.Linear(2 * self.decoder_hidden_size + self.encoder_hidden_size, self.encoder_hidden_size),
            nn.Tanh(),
            nn.Linear(self.encoder_hidden_size, 1)
        )
        self.lstm = nn.LSTM(input_size=self.out_feats, hidden_size=self.decoder_hidden_size)
        self.fc = nn.Linear(self.encoder_hidden_size + self.out_feats, self.out_feats)
        self.fc_out = nn.Linear(self.decoder_hidden_size + self.encoder_hidden_size, self.out_feats)
        self.fc.weight.data.normal_()
        self.edge_index = edge_index
    

    def forward(self, input_encoded: torch.Tensor, y_history: torch.Tensor, sample: torch.Tensor):
        """
        Perform forward computation.

        Args:
            input_encoded: (torch.Tensor): tensor of encoded input
            y_history: (torch.Tensor): shifted target
        """
        h_t, c_t = (init_hidden(input_encoded, self.decoder_hidden_size), init_hidden(input_encoded, self.decoder_hidden_size))
        context = Variable(torch.zeros(input_encoded.size(0), self.encoder_hidden_size))

        a = self.edge_index[0][sample]
        b = self.edge_index[1][sample]
        sample_edge_index = torch.stack([a, b], dim=0)
        mapped_arr = map_values(sample_edge_index, (sample_edge_index.min(), sample_edge_index.max()))

        for t in range(self.seq_len):
            x = torch.cat((h_t.repeat(self.seq_len, 1, 1).permute(1, 0, 2),
                           c_t.repeat(self.seq_len, 1, 1).permute(1, 0, 2),
                           input_encoded.to(device)), dim=2)
            x = tf.softmax(
                self.attn(
                    x.view(-1, 2 * self.decoder_hidden_size + self.encoder_hidden_size)
                ).view(-1, self.seq_len),
                dim=1)
            context = torch.bmm(x.unsqueeze(1), input_encoded.to(device))[:, 0, :]  # (batch_size, encoder_hidden_size)
            y_tilde = self.fc(torch.cat((context.to(device), y_history[:, t].to(device)),
                                        dim=1))  # (batch_size, out_size)            
            self.lstm.flatten_parameters()
            _, (h_t, c_t) = self.lstm(y_tilde.unsqueeze(0), (h_t, c_t))
            h_t = self.gcn_d(h_t,mapped_arr.to(device))
        

        return self.fc_out(torch.cat((h_t[0], context.to(device)), dim=1))  # predicting value at t=self.seq_length+1


class AutoEncForecast(nn.Module):
    def __init__(self, input_att,temporal_att,hidden_size_encoder,seq_len,denoising,directions,hidden_size_decoder,input_size,output_size,sample_size,edge_index,spatial_matrix,use_spatial=False):
        """
        Initialize the network.

        Args:
            config:
            input_size: (int): size of the input
        """
        super(AutoEncForecast, self).__init__()
        self.encoder = AttnEncoder(hidden_size_encoder,seq_len,denoising,directions,input_size,edge_index,spatial_matrix,use_spatial).to(device) if input_att else \
            Encoder(hidden_size_encoder, seq_len, input_size,edge_index).to(device)
        self.decoder = AttnDecoder(seq_len,hidden_size_encoder,hidden_size_decoder,output_size,edge_index).to(device) if temporal_att else Decoder(seq_len,hidden_size_decoder,output_size,edge_index).to(device)

        self.fc1 = nn.Linear(256,1)#.cuda()
        self.tanh = nn.Tanh()#.cuda()
        self.sigmoid = nn.Sigmoid()
        self.sample_size = sample_size

    def calcPerm(self,l: list, m: int):
        for i in permutations(l, m):
            yield list((i))
    
    def forward(self, encoder_input: torch.Tensor, y_hist: torch.Tensor, sample: torch.Tensor ,return_attention: bool = False):
        """
        Forward computation. encoder_input_inputs.

        Args:
            encoder_input: (torch.Tensor): tensor of input data
            y_hist: (torch.Tensor): shifted target
            return_attention: (bool): whether or not to return the attention
        """
        attentions, encoder_output = self.encoder(encoder_input, sample)   

        outputs = self.decoder(encoder_output, y_hist.float(), sample)
        g = torch.zeros(outputs.shape[1],outputs.shape[1])#.cuda()
        outputs = outputs.transpose(1,0)
        idx1 = 0
        idx2 = 0
        for i,j in self.calcPerm(outputs, 2):
                a = self.fc1(i)
                b = self.fc1(j)
                c = self.sigmoid(self.tanh(a+b))#.cuda()
                g[idx1,idx2] = c
                if((idx2+1)%self.sample_size):
                    idx2+=1
                else:
                    idx2 = 0
                    idx1+= 1
        if return_attention:
            return outputs, attentions,g
        return outputs,g

In [11]:
# target_index = 252
target_index = 0

In [13]:
# distance = np.load('C:/Users/psheth5/STCD-RL/data/data/watershed.npy')
# distance = distance[:, target_index]

# spatial_matrix = []

# for d in distance:
#     if np.isfinite(d):
#         spatial_matrix.append(1 / (d + 1))
#     else:
#         spatial_matrix.append(0)

# spatial_matrix1 = torch.FloatTensor(spatial_matrix).cuda()
# torch.max(spatial_matrix1)

spatial_matrix1 = torch.FloatTensor(adj_mat).cuda()
torch.max(spatial_matrix1)

tensor(1., device='cuda:0')

In [14]:
sampling_size = 100
model = AutoEncForecast(input_att=True,temporal_att=True,hidden_size_encoder=64,seq_len=30,denoising=False,directions=1,hidden_size_decoder=64,input_size=sampling_size,output_size=sampling_size,sample_size=sampling_size,edge_index=edge_index,spatial_matrix = spatial_matrix1,use_spatial=True).to(device)

In [15]:
import random
def update_graph(sample,inferred_cg,g):
    for i in range(len(sample)):
        for j in range(len(sample)):
            inferred_cg[sample[i],sample[j]] += g[i,j].cpu()
    return inferred_cg

def update_attention(sample,att,attention):
    temp = torch.mean(att,axis=1)
    for i in range(len(sample)):
        for j in range(len(sample)):
            attention[sample[i],sample[j]] += temp[i,j].cpu()
    return attention

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def get_num_cycles(gra):
    # gra = gra.cpu().detach().numpy()
    G = nx.from_numpy_matrix(gra)
    H = G.to_directed()
    try:
        return torch.FloatTensor([len(list(nx.find_cycle(H, orientation="original")))])#.cuda()
    except:
        return 0

def is_dag(gr):
    # gr = gr.cpu().detach().numpy()
    G = nx.from_numpy_matrix(gr)
    return int(nx.is_directed_acyclic_graph(G))

In [16]:
## helper functions
def calc_discounted_rewards(rewards, gamma):
    ''' 
    Simple implementation for better understanding
    gets rewards of an entire episode and calculates R_t for every t
    '''
    
    returns = []
    
    for t in range(len(rewards)):
        ret = 0
        
        for t_p in range(t, len(rewards)):
            ret += gamma ** (t_p - t) * rewards[t_p]
            
        returns.insert(0, ret)
        
    return returns

In [17]:
random.seed(26)
np.random.seed(26)
torch.manual_seed(26)
torch.cuda.manual_seed(26)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
global_step, logging_loss = 0, 0.0
train_loss = 0.0
reg1= True
reg2= True
reg_factor1= 1e-2
reg_factor2= 1e-2
gradient_accumulation_steps = 1
max_grad_norm = 0.1
logging_steps = 100
criterion = nn.BCELoss()
crit_mse = nn.MSELoss()
crit_nll = nn.NLLLoss()
save_steps = 5000
eval_during_training = False
output_dir = "Models"
lrs_step_size=5000
do_eval=True
reg_factor = 1
BASELINE_REWARD = 20
breakout = 0
inferred_cg = np.zeros((n_features,n_features))
attention = np.zeros((n_features,n_features))
min_loss = 1e+5
for epoch in tqdm(range(50), unit="epoch"):
    attentions = []
    outputs = []
    total_loss = 0
    loss_1 = []
    loss_2 = []
    for i, batch in tqdm(enumerate(train_iter), total=len(train_iter), unit="batch"):
        original_list = list(range(3129))
        while(len(original_list)!=0):
            sampled_list = []
            optimizer.zero_grad()
            model.train()
            feature, y_hist, target = batch
            try:
                sample = random.sample(original_list, sampling_size)
            except:
                sample = random.sample(original_list,len(original_list))
                sample.extend(random.sample(range(feature.shape[2]),sampling_size-len(original_list)-1))
                sample.append(0)
            # add the sampled elements to the sampled list
            sampled_list.extend(sample)
            # remove the sampled elements from the original list
            original_list = [element for element in original_list if element not in sample]
            feature = feature[:,:,sample]
            y_hist = y_hist[:,:,sample]
            target = target[:,sample]
            
            output,att,g = model(feature.to(device), y_hist.to(device),sample,return_attention=True)
            temp = torch.mean(att,axis=1)
            output = output.reshape(output.shape[1],output.shape[0])
            inferred_cg = update_graph(sample,inferred_cg,g)
            attention = update_attention(sample,att,attention)
            num_params = count_parameters(model)
            #target = target.cuda()
            loss2 = crit_mse(output,target)

            n = torch.FloatTensor([feature.shape[0]])#.cuda()
            loss = -1*(n*torch.log(loss2) + reg_factor1*get_num_cycles(inferred_cg) + reg_factor2*is_dag(inferred_cg))
            loss_2.append(loss2)
            loss_1.append(loss.item())
            if(len(original_list)%1000==0):
                print("The number of nodes sampled are ", 3129 - len(original_list))
    loss_tot = 0
    loss_21 = calc_discounted_rewards(loss_2,0.98)
    for indexe in range(len(loss_21)):
        loss_tot += loss_1[indexe] * torch.log(loss_21[indexe])
    loss_tot = (loss_tot - BASELINE_REWARD) / len(loss_21)
    loss_tot.backward()
    optimizer.step()
    total_loss +=loss_tot.item()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    if(round(total_loss/(i+1),2)<=round(min_loss/(i+1),2)):
        min_loss = total_loss
        breakout = 0
        torch.save(model.state_dict(),os.getcwd()+"/Models/MyModel_GCN_FullModel_"+str(epoch))
    else:
        breakout+=1
    print("Loss is ", total_loss, " for epoch ", epoch)
    print(np.where(attention[target_index,:]!=0),len(np.where(attention[target_index,:]!=0)[0]),get_num_cycles(inferred_cg),is_dag(inferred_cg))

 25%|██▌       | 1/4 [01:12<03:38, 72.77s/batch]

The number of nodes sampled are  3129



 50%|█████     | 2/4 [03:14<03:23, 101.68s/batch]

The number of nodes sampled are  3129



 75%|███████▌  | 3/4 [05:52<02:07, 127.19s/batch]

...

The number of nodes sampled are  3129



 50%|█████     | 2/4 [1:14:47<1:14:36, 2238.04s/batch]

The number of nodes sampled are  3129


 32%|███▏      | 16/50 [23:19:12<49:33:18, 5247.00s/epoch]


KeyboardInterrupt: 

In [26]:
at1 = frame.columns[attention[target_index,:].argsort()[-100:]]
c=0
at2=[]
for i in at1:
    if(float(i.split()[1])<32):
        at2.append(i)
        c+=1
    if(c==30):
        break
print(at2)
plot_graph(at2,"mapTestFT1.html")


['-96.531250  30.843750\n', '-97.531250  31.156250\n', '-96.343750  29.843750\n', '-97.468750  31.406250\n', '-97.468750  30.656250\n', '-96.218750  31.593750\n', '-96.593750  29.968750\n', '-97.718750  31.343750\n', '-96.593750  30.031250\n', '-97.468750  31.531250\n', '-96.656250  31.156250\n', '-96.968750  30.468750\n', '-95.531250  29.156250\n', '-96.406250  30.718750\n', '-97.156250  31.406250\n', '-95.843750  29.406250\n', '-96.656250  31.656250\n', '-96.281250  29.906250\n', '-97.781250  30.968750\n', '-97.031250  30.218750\n', '-96.968750  31.593750\n', '-96.593750  30.656250\n', '-96.031250  30.593750\n', '-97.656250  30.531250\n', '-96.281250  30.843750\n', '-97.468750  31.468750\n', '-95.968750  30.093750\n', '-96.468750  30.531250\n', '-97.781250  31.968750\n', '-97.593750  30.906250\n']


In [38]:
te12 = frame.columns[[0, 4, 6, 7, 9, 11, 12, 13, 16, 17, 19, 22, 24, 26, 28, 29, 32, 33, 34, 47, 51, 52, 53, 54, 56, 58, 60, 61, 63, 64, 65, 66, 69, 71, 72, 76, 80, 81, 83, 85, 88, 89, 90, 94, 95, 96, 97, 98, 99, 103, 104, 107, 108, 109, 114, 117, 118, 126, 127, 129, 132, 134, 137, 138, 140, 141, 144, 146, 149, 150, 153, 156, 157, 158, 162, 163, 169, 170, 172, 175, 176, 181, 182, 184, 185, 186, 187, 189, 191, 193, 198, 199, 201, 202, 209, 210, 211, 212, 214, 216, 217, 218, 224, 232, 233, 235, 236, 239, 240, 241, 242, 244, 245, 246, 248, 250, 251, 253, 255, 259, 260, 261, 263, 266, 267, 268, 269, 271, 272, 275, 276, 278, 279, 281, 283, 285, 287, 288, 289, 292, 295, 297, 298, 302, 303, 304, 305, 306, 307, 308, 311, 312, 314, 315, 320, 321, 325, 326, 328, 331, 332, 333, 334, 335, 336, 339, 340, 343, 344, 345, 349, 352, 353, 358, 359, 360, 366, 375, 379, 380, 383, 384, 387, 389, 393, 395, 396, 399, 401, 403, 405, 406, 407, 410, 411, 413, 414, 416, 417, 420, 422, 425, 427, 430, 431, 433, 434, 436, 438, 440, 449, 450, 452, 457, 459, 460, 461, 462, 464, 465, 468, 469, 471, 472, 473, 474, 479, 480, 482, 483, 486, 488, 489, 492, 494, 496, 499, 500, 502, 504, 508, 509, 511, 515, 517, 518, 521, 523, 524, 526, 527, 529, 530, 532, 536, 539, 543, 558, 559, 563, 564, 569, 571, 573, 575, 576, 584, 585, 591, 592, 595, 598, 600, 608, 609, 614, 615, 619, 622, 626, 628, 630, 631, 634, 637, 638, 640, 641, 642, 643, 644, 645, 646, 647, 650, 652, 654, 655, 658, 661, 662, 663, 664, 665, 666, 667, 668, 669, 672, 673, 676, 680, 685, 686, 688, 691, 696, 697, 701, 705, 706, 708, 712, 713, 718, 723, 728, 730, 732, 733, 734, 736, 738, 743, 744, 745, 747, 749, 750, 752, 753, 754, 755, 756, 759, 761, 762, 763, 765, 766, 767, 768, 769, 770, 771, 772, 773, 777, 779, 783, 786, 787, 793, 795, 796, 797, 798, 800, 803, 805, 806, 807, 809, 811, 814, 816, 818, 820, 821, 822, 823, 824, 825, 826, 832, 833, 835, 836, 838, 840, 841, 843, 844, 846, 849, 850, 853, 857, 858, 861, 864, 867, 868, 870, 874, 875, 877, 878, 882, 884, 888, 889, 891, 893, 895, 896, 897, 898, 899, 900, 901, 902, 905, 910, 913, 915, 917, 919, 921, 923, 925, 928, 930, 932, 933, 939, 944, 945, 946, 952, 953, 954, 955, 956, 958, 964, 965, 966, 967, 968, 969, 970, 972, 978, 980, 981, 984, 989, 992, 993, 1003, 1004, 1006, 1009, 1010, 1011, 1016, 1018, 1021, 1022, 1025, 1027, 1028, 1033, 1034, 1037, 1041, 1043, 1045, 1046, 1047, 1049, 1052, 1055, 1057, 1058, 1059, 1062, 1063, 1066, 1067, 1069, 1071, 1075, 1078, 1079, 1081, 1082, 1083, 1085, 1087, 1088, 1090, 1094, 1096, 1097, 1098, 1099, 1103, 1104, 1105, 1106, 1107, 1114, 1115, 1119, 1120, 1124, 1125, 1126, 1127, 1128, 1130, 1131, 1132, 1133, 1143, 1144, 1145, 1146, 1148, 1150, 1151, 1153, 1154, 1157, 1158, 1161, 1163, 1164, 1165, 1170, 1171, 1177, 1179, 1183, 1185, 1188, 1191, 1193, 1194, 1195, 1197, 1201, 1202, 1204, 1207, 1208, 1213, 1214, 1215, 1226, 1227, 1230, 1232, 1233, 1234, 1236, 1237, 1239, 1241, 1247, 1248, 1252, 1254, 1255, 1256, 1257, 1258, 1259, 1262, 1266, 1268, 1270, 1274, 1275, 1277, 1278, 1283, 1286, 1288, 1290, 1291, 1295, 1296, 1298, 1299, 1300, 1303, 1306, 1307, 1310, 1311, 1312, 1316, 1320, 1323, 1324, 1328, 1329, 1332, 1334, 1335, 1336, 1339, 1340, 1341, 1344, 1351, 1352, 1354, 1356, 1358, 1361, 1362, 1365, 1367, 1368, 1369, 1372, 1373, 1378, 1380, 1383, 1384, 1386, 1388, 1394, 1395, 1397, 1398, 1402, 1403, 1404, 1406, 1414, 1417, 1419, 1420, 1421, 1424, 1425, 1428, 1430, 1433, 1434, 1437, 1438, 1439, 1442, 1444, 1445, 1449, 1451, 1453, 1459, 1465, 1473, 1475, 1479, 1481, 1482, 1483, 1486, 1488, 1489, 1490, 1491, 1492, 1493, 1494, 1497, 1499, 1500, 1501, 1503, 1504, 1505, 1506, 1510, 1513, 1514, 1519, 1520, 1526, 1529, 1532, 1534, 1538, 1539, 1540, 1542, 1544, 1545, 1546, 1547, 1548, 1550, 1552, 1559, 1562, 1563, 1564, 1565, 1568, 1570, 1571, 1574, 1576, 1577, 1578, 1579, 1580, 1581, 1582, 1584, 1586, 1588, 1589, 1594, 1595, 1596, 1599, 1600, 1601, 1608, 1609, 1612, 1613, 1614, 1615, 1616, 1617, 1619, 1620, 1621, 1629, 1630, 1631, 1632, 1633, 1634, 1635, 1636, 1645, 1646, 1650, 1651, 1652, 1653, 1658, 1660, 1662, 1664, 1665, 1671, 1674, 1675, 1676, 1678, 1679, 1680, 1684, 1688, 1691, 1692, 1693, 1694, 1695, 1696, 1697, 1698, 1699, 1702, 1703, 1704, 1706, 1707, 1709, 1710, 1714, 1715, 1718, 1720, 1721, 1722, 1724, 1725, 1727, 1729, 1731, 1734, 1735, 1738, 1739, 1740, 1741, 1742, 1743, 1745, 1748, 1749, 1750, 1752, 1754, 1756, 1758, 1759, 1760, 1762, 1763, 1765, 1767, 1769, 1771, 1774, 1775, 1779, 1780, 1783, 1784, 1786, 1788, 1791, 1793, 1794, 1796, 1800, 1801, 1802, 1804, 1807, 1808, 1809, 1810, 1812, 1817, 1818, 1819, 1820, 1821, 1822, 1826, 1827, 1830, 1832, 1835, 1837, 1838, 1841, 1844, 1845, 1847, 1849, 1851, 1852, 1854, 1855, 1857, 1860, 1862, 1865, 1866, 1867, 1869, 1870, 1875, 1876, 1877, 1879, 1882, 1885, 1887, 1892, 1893, 1895, 1897, 1898, 1899, 1901, 1902, 1903, 1905, 1907, 1908, 1909, 1912, 1913, 1914, 1917, 1921, 1926, 1937, 1938, 1939, 1941, 1945, 1949, 1950, 1951, 1953, 1957, 1958, 1959, 1961, 1963, 1964, 1965, 1969, 1971, 1975, 1976, 1977, 1978, 1979, 1984, 1985, 1986, 1987, 1989, 1990, 1991, 1993, 1995, 1997, 2004, 2007, 2008, 2009, 2010, 2012, 2013, 2017, 2018, 2019, 2020, 2022, 2023, 2026, 2028, 2029, 2030, 2031, 2033, 2034, 2036, 2037, 2038, 2040, 2043, 2046, 2049, 2050, 2056, 2057, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2072, 2073, 2083, 2086, 2091, 2092, 2096, 2099, 2100, 2101, 2103, 2104, 2107, 2108, 2111, 2112, 2116, 2117, 2118, 2119, 2120, 2121, 2126, 2127, 2130, 2131, 2132, 2133, 2140, 2141, 2142, 2145, 2146, 2149, 2150, 2153, 2154, 2155, 2157, 2161, 2162, 2163, 2165, 2166, 2169, 2170, 2172, 2174, 2175, 2177, 2179, 2185, 2187, 2190, 2192, 2194, 2195, 2196, 2197, 2198, 2200, 2201, 2205, 2206, 2212, 2215, 2217, 2220, 2222, 2224, 2226, 2230, 2232, 2233, 2235, 2236, 2237, 2239, 2241, 2242, 2243, 2244, 2249, 2251, 2252, 2253, 2256, 2258, 2259, 2260, 2262, 2263, 2264, 2265, 2270, 2275, 2280, 2283, 2284, 2286, 2289, 2292, 2293, 2294, 2295, 2296, 2302, 2303, 2305, 2307, 2309, 2313, 2316, 2317, 2318, 2321, 2323, 2326, 2327, 2328, 2331, 2336, 2338, 2339, 2341, 2342, 2344, 2349, 2350, 2351, 2352, 2354, 2359, 2363, 2365, 2369, 2371, 2375, 2376, 2378, 2381, 2384, 2388, 2392, 2396, 2398, 2401, 2402, 2403, 2405, 2406, 2407, 2408, 2411, 2412, 2414, 2415, 2417, 2418, 2420, 2422, 2425, 2429, 2430, 2438, 2439, 2440, 2444, 2446, 2452, 2453, 2454, 2455, 2456, 2457, 2458, 2459, 2463, 2465, 2466, 2473, 2474, 2475, 2476, 2479, 2480, 2481, 2484, 2487, 2488, 2489, 2494, 2497, 2499, 2500, 2501, 2502, 2504, 2505, 2508, 2510, 2512, 2514, 2518, 2519, 2521, 2526, 2530, 2532, 2537, 2539, 2541, 2547, 2548, 2550, 2551, 2552, 2554, 2556, 2557, 2558, 2562, 2564, 2566, 2569, 2570, 2576, 2578, 2580, 2583, 2586, 2587, 2592, 2593, 2594, 2595, 2596, 2601, 2609, 2610, 2612, 2614, 2616, 2617, 2623, 2625, 2629, 2632, 2634, 2636, 2637, 2642, 2646, 2647, 2648, 2652, 2654, 2658, 2659, 2660, 2661, 2662, 2663, 2664, 2665, 2666, 2667, 2668, 2670, 2672, 2673, 2676, 2678, 2679, 2682, 2686, 2687, 2688, 2689, 2690, 2695, 2697, 2698, 2699, 2700, 2703, 2708, 2711, 2713, 2715, 2720, 2721, 2722, 2726, 2728, 2729, 2730, 2731, 2733, 2734, 2738, 2740, 2746, 2748, 2749, 2750, 2751, 2752, 2753, 2754, 2755, 2760, 2767, 2771, 2773, 2777, 2778, 2782, 2784, 2785, 2787, 2789, 2791, 2796, 2797, 2798, 2799, 2800, 2802, 2807, 2810, 2811, 2812, 2814, 2815, 2817, 2818, 2819, 2820, 2821, 2823, 2824, 2829, 2832, 2833, 2834, 2835, 2836, 2839, 2840, 2841, 2843, 2848, 2851, 2857, 2858, 2859, 2860, 2865, 2866, 2867, 2868, 2870, 2871, 2872, 2880, 2881, 2885, 2886, 2887, 2895, 2898, 2901, 2905, 2907, 2908, 2910, 2911, 2913, 2915, 2918, 2920, 2921, 2922, 2927, 2929, 2932, 2933, 2936, 2938, 2939, 2943, 2944, 2947, 2948, 2950, 2953, 2960, 2961, 2962, 2963, 2967, 2972, 2974, 2978, 2979, 2980, 2981, 2982, 2983, 2986, 2989, 2990, 2994, 2997, 2998, 3000, 3005, 3006, 3007, 3008, 3009, 3010, 3011, 3015, 3016, 3018, 3019, 3021, 3022, 3024, 3026, 3027, 3028, 3030, 3031, 3032, 3041, 3044, 3045, 3048, 3052, 3054, 3055, 3057, 3062, 3063, 3065, 3067, 3073, 3074, 3082, 3083, 3084, 3085, 3090, 3092, 3095, 3098, 3104, 3107, 3111, 3112, 3114, 3116, 3120, 3121]]
te213 = []
c=0
for i in te12:
    if(float(i.split()[1])<32):
        te213.append(i)
        c+=1
    if(c==30):
        break
print(len(te213))
plot_graph(te213, "mapTestFT1.html")

30


In [ ]:
index_list = list(np.where(attention[target_index,:]>np.mean(attention[target_index,:]))[0])
cols = list(frame.columns[index_list])
with open("index_list.txt", "w") as output:
    output.write(str(index_list))
with open("cols.txt", "w") as output:
    output.write(str(cols))

In [37]:
def plot_graph(a_new,fname):
    latitude = []
    longitude = []
    for j,i in enumerate(a_new):
            latitude.append(float(i.split()[0]))
            longitude.append(float(i.split()[1]))
    fig = go.Figure()
    fig.add_trace(go.Scattermapbox(
        mode = "markers",
        lat = longitude,
        lon = latitude,
        marker = { 'color':'red',
                    'size': 10}
    ))

    fig.update_layout(margin={'l':0,'t':0,'b':0,'r':0},
                    mapbox={
                        'center':{'lon':139,'lat':36.5},
                        'style': "stamen-terrain",
                        'zoom' : 4.5},
                    width=1600,
                    height=900,)
    fig.show()
    fig.write_html(fname)

In [ ]:
test = pd.DataFrame()
x = attention[target_index,:].argsort()[-30:][::-1]
test['locations'] = frame.columns[x]
test['attention'] = attention[target_index,x]
test['longitude'] = test['locations'].apply(lambda x: float(x.split()[0]))
test['latitude'] = test['locations'].apply(lambda x: float(x.split()[1]))
test['attention'][0] = max(test['attention'][1:])
test = test.loc[:,['longitude','latitude','attention']]
# test['attention'] = (test['attention'] - test['attention'].min()) / (test['attention'].max() - test['attention'].min())
test['attention'] = test['attention']/test['attention'].sum()
# test.loc[len(test.index)] = [-97.843750, 32.531250, 0.6] 
test


In [ ]:
import plotly.express as px
fig = px.density_mapbox(test, lat='latitude', lon='longitude', z='attention', radius=10,
                        center=dict(lat=0, lon=180), zoom=0,
                        color_continuous_scale=[[0.0, 'rgb(255, 0, 0)'],[0.2, 'rgb(255, 0, 0)'],[0.4, 'rgb(255, 255, 0)'],[0.6, 'rgb(0, 255, 0)'],[0.8, 'rgb(0, 255, 255)'],[1.0, 'rgb(0, 0, 255)']],
                        mapbox_style="stamen-terrain")
fig.show()
fig.write_html("mapTestWS1.html")

In [ ]:
frame.columns[target_index]

In [ ]:
x = attention[target_index,:].argsort()[-30:][::-1]
# y = inferred_cg[:,1200].argsort()[-30:][::-1]
# plot_graph(frame.columns[y],"GCNgraph_inf1.html")
plot_graph(frame.columns[x],"GCNgraph_att1.html")

In [ ]:
frame.columns[x]

In [ ]:
t = attention[:,1200]
inds = np.where(t>np.mean(t))[0]
sorted_indexes = inds[np.argsort(t[inds])[::-1]]
sorted_indexes[0:30]
plot_graph(frame.columns[sorted_indexes[0:30]],"GCNgraph_inf2.html")

In [ ]:
import numpy as np

# create an example array
arr = inferred_cg[:,1200]

# initialize an empty list to store the indexes of the top 30 unique values
top_indexes = []

# iterate over the array and find the indexes of the top 30 unique values
for i in range(30):
    # find the index of the maximum value that hasn't already been added to the list
    max_index = np.argmax(arr)
    while max_index in top_indexes:
        arr[max_index] = -np.inf  # set the maximum value to -inf so it won't be selected again
        max_index = np.argmax(arr)
    
    # add the current index to the top_indexes list
    top_indexes.append(max_index)

candidates = frame.columns[top_indexes]
target = frame.columns[1200]



In [ ]:
from geopy.distance import distance

# example list of latitude longitude coordinates
coords_str = frame.columns[x]

# target latitude longitude
target_lat = float(target.split()[1])
target_lon = float(target.split()[0])

# convert coordinates to float values and calculate distances from target
coords = [tuple(map(float, c.split()[::-1])) for c in coords_str]
distances = [distance((target_lat, target_lon), c).km for c in coords]

# determine which points are above and below the target and output index and 1 or -1
for i, d in enumerate(distances):
    if coords[i][0] >= target_lat:
        print(f"Point {i}: 1")
    else:
        print(f"Point {i}: -1")


In [ ]:
coords[0]

In [ ]:
import pandas as pd

data = pd.read_csv("C:/Users/psheth5/STCD-RL/data/data/StandardizedData.csv").columns[3000:3129]
x = pd.DataFrame()
x['longitude'] = data.str.split().str[0]
x['latitude'] = data.str.split().str[1]
x['longitude'] = x['longitude'].astype(float)
x['latitude'] = x['latitude'].astype(float)

x.to_csv("lonlatinfo7.txt",index=False,header=False,sep=",")

In [ ]:
elevate = pd.DataFrame()

In [ ]:
x = pd.read_csv("C:/Users/psheth5/STCD-RL/data/data/bulk-pqs (6).csv",header=None)
elevate = pd.concat([elevate,x],axis=0)
print(elevate.shape)

In [ ]:
# Rows to be dropped
rows_to_drop = [501, 1002, 1503, 2004, 2505, 3006]

# Drop specific rows
elevate = elevate.drop(elevate.index[rows_to_drop])

# Reset the index if needed
elevate = elevate.reset_index(drop=True)

In [ ]:
import numpy as np

# Check the modified dataframe
elevations = elevate[3].values

# Create an empty adjacency matrix
adjacency_matrix = np.zeros((len(elevations), len(elevations)))

# Populate the adjacency matrix based on the condition
for i in range(len(elevations)):
    for j in range(len(elevations)):
        if elevations[i] > elevations[j]:
            adjacency_matrix[i, j] = 1

# Check the adjacency matrix
print(adjacency_matrix)

In [ ]:
np.save("ElevationAdjacency.npy",adjacency_matrix)